In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# Набор данных взят с https://www.kaggle.com/aungpyaeap/fish-market
# Параметры нескольких популярных промысловых рыб
# length 1 = Body height
# length 2 = Total Length
# length 3 = Diagonal Length
fish_data = pd.read_csv("datasets/Fish.csv", delimiter=',')
print(fish_data)

    Species  Weight  Length1  Length2  Length3   Height   Width
0     Bream   242.0     23.2     25.4     30.0  11.5200  4.0200
1     Bream   290.0     24.0     26.3     31.2  12.4800  4.3056
2     Bream   340.0     23.9     26.5     31.1  12.3778  4.6961
3     Bream   363.0     26.3     29.0     33.5  12.7300  4.4555
4     Bream   430.0     26.5     29.0     34.0  12.4440  5.1340
..      ...     ...      ...      ...      ...      ...     ...
154   Smelt    12.2     11.5     12.2     13.4   2.0904  1.3936
155   Smelt    13.4     11.7     12.4     13.5   2.4300  1.2690
156   Smelt    12.2     12.1     13.0     13.8   2.2770  1.2558
157   Smelt    19.7     13.2     14.3     15.2   2.8728  2.0672
158   Smelt    19.9     13.8     15.0     16.2   2.9322  1.8792

[159 rows x 7 columns]


In [3]:
# Выделим входные параметры и целевое значение
x_labels = ['Height', 'Width']
y_label = 'Weight'

data = fish_data[x_labels + [y_label]]
print(data)

      Height   Width  Weight
0    11.5200  4.0200   242.0
1    12.4800  4.3056   290.0
2    12.3778  4.6961   340.0
3    12.7300  4.4555   363.0
4    12.4440  5.1340   430.0
..       ...     ...     ...
154   2.0904  1.3936    12.2
155   2.4300  1.2690    13.4
156   2.2770  1.2558    12.2
157   2.8728  2.0672    19.7
158   2.9322  1.8792    19.9

[159 rows x 3 columns]


In [4]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

32


In [5]:
# Генерируем уникальный seed
my_code = "Шумилин"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Выделим обучающую, валидационную и тестовую выборки
train_x = train[x_labels]
train_y = np.array(train[y_label]).reshape(-1,1)

val_x = val[x_labels]
val_y = np.array(val[y_label]).reshape(-1,1)

test_x = test[x_labels]
test_y = np.array(test[y_label]).reshape(-1,1)

In [8]:
# Нормируем значения параметров
scaler_x = MinMaxScaler()
scaler_x.fit(train_x)
scaled_train_x = scaler_x.transform(train_x)

scaler_y = MinMaxScaler()
scaler_y.fit(train_y)
scaled_train_y = scaler_y.transform(train_y)

In [9]:
# Создадим модель метода k-ближайших соседей и обучим ее на нормированных данных. По умолчанию k = 5.
minmse=10
mink=0
a=[]
scaled_val_x = scaler_x.transform(val_x)
scaled_val_y = scaler_y.transform(val_y)
for k in range(1,51):
            model1 = KNeighborsRegressor(n_neighbors = k)
            model1.fit(scaled_train_x, scaled_train_y)
            val_predicted = model1.predict(scaled_val_x)
            mse1 = mean_squared_error(scaled_val_y, val_predicted)
            a.append(mse1)
            if mse1<minmse:
                minmse=mse1
                mink=k

In [10]:
print("мин. сред-квадрат. ошибка",minmse)
print("значение k, которому соответсвует мин. сред-квадрат. ошибка",mink)
print()
print(a)

мин. сред-квадрат. ошибка 0.002691614118538752
значение k, которому соответсвует мин. сред-квадрат. ошибка 23

[0.01611062622056466, 0.015957125451066083, 0.008101790523472015, 0.005526502433823582, 0.00399658667883363, 0.0033660742300411486, 0.0037388387176110027, 0.0037822329787012034, 0.003678186859437452, 0.0038900287252668416, 0.003968392930197483, 0.0038243522483823473, 0.003991164714298776, 0.004389408523140179, 0.004866814026266363, 0.004451261222438786, 0.004451811989981475, 0.0038932705306567172, 0.0034597412490361013, 0.0032084801741550546, 0.002972904850197053, 0.002806391488543501, 0.002691614118538752, 0.002773968046197499, 0.002937895578073813, 0.002922119614068264, 0.002921837545216096, 0.0031453485145110658, 0.0032848671655969592, 0.003247038968224496, 0.00329875877508733, 0.003402911228797016, 0.0034592711331543295, 0.003617550248810565, 0.003736291293922486, 0.0036018338658321764, 0.003415325836750479, 0.003499497191140977, 0.003595034458472183, 0.0037825277949891274

In [11]:
model1 = KNeighborsRegressor(n_neighbors = 8)
model1.fit(scaled_train_x, scaled_train_y)
val_predicted = model1.predict(scaled_val_x)
mse1 = mean_squared_error(scaled_val_y, val_predicted)
print(mse1)

0.0037822329787012034


In [12]:
# Проверим результат на тестевойой выборке.
scaled_test_x = scaler_x.transform(test_x)
scaled_test_y = scaler_y.transform(test_y)

test_predicted = model1.predict(scaled_test_x)

mse2 = mean_squared_error(scaled_test_y,test_predicted)
print(mse2)

0.01400516319675369
